In [1]:
import csv
import Preprocessing as ppy

In [2]:
def openFileCsv(namaFile):
    data_hasil = []
    with open(namaFile, mode='r', encoding='utf8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            data_hasil.append(row)
            line_count += 1
        print(f'Processed {line_count} lines.')
    return data_hasil

In [3]:
def getData():
    data = openFileCsv('data_training.csv')
    newData = []
    for i in range(len(data)):
        newData.append([ppy.generateTermInSentence(data[i]["Teks"]), data[i]["label"]])
    return newData
print(getData())

Processed 914 lines.
[['kfc yg deket enhaii dy', '0'], ['maaf janji penuh janji', '0'], ['bunga ato coklat min', '0'], ['sambl nunggu gimana kalo ngerjain form formnya aja', '0'], ['akademik perhati tuk jadwal kontrak kuliah ang sebenarny kuliah jadwal kur yg ang ato jadwal yg jadwal bayang tuk tuju kontrak kuliah ga laku tuk mk yg jalan ang spt skripsi sidang harap perhati', '0'], ['info pla assalam yg ambil mk pla wajib kumpul kamis agustus wibb agenda bekal pla gdg jica lt r s informasiny harap perhati d sebar', '0'], ['info teman yg kumpul data asisten bukti upa surat terang ambil andri', '0'], ['daftar langgan tcash full service tingkat saldo dg isi saldo tcash merchant logo tcash', '0'], ['pin tcash silah pin nikmat layan tcash telkomsel', '0'], ['mad bingung bkn lapor', '0'], ['aamiin makasih moga cepat lulus', '0'], ['aamiin makasih moga teman cepat nyusul moga urus lancar allah aamiin', '0'], ['abis maghrib yaa', '0'], ['warkop deket indomaret gum', '0'], ['bip studio foto git

In [4]:
term = ppy.getAllTerm()
print(term)

Processed 914 lines.
['kfc', 'yg', 'deket', 'enhaii', 'dy', 'maaf', 'janji', 'penuh', 'bunga', 'ato', 'coklat', 'min', 'sambl', 'nunggu', 'gimana', 'kalo', 'ngerjain', 'form', 'formnya', 'aja', 'akademik', 'perhati', 'tuk', 'jadwal', 'kontrak', 'kuliah', 'ang', 'sebenarny', 'kur', 'bayang', 'tuju', 'ga', 'laku', 'mk', 'jalan', 'spt', 'skripsi', 'sidang', 'harap', 'info', 'pla', 'assalam', 'ambil', 'wajib', 'kumpul', 'kamis', 'agustus', 'wibb', 'agenda', 'bekal', 'gdg', 'jica', 'lt', 'r', 's', 'informasiny', 'd', 'sebar', 'teman', 'data', 'asisten', 'bukti', 'upa', 'surat', 'terang', 'andri', 'daftar', 'langgan', 'tcash', 'full', 'service', 'tingkat', 'saldo', 'dg', 'isi', 'merchant', 'logo', 'pin', 'silah', 'nikmat', 'layan', 'telkomsel', 'mad', 'bingung', 'bkn', 'lapor', 'aamiin', 'makasih', 'moga', 'cepat', 'lulus', 'nyusul', 'urus', 'lancar', 'allah', 'abis', 'maghrib', 'yaa', 'warkop', 'indomaret', 'gum', 'bip', 'studio', 'foto', 'gitu', 'nama', 'tpi', 'gatau', 'msh', 'fungsi', 'en

In [5]:
import re

def rowTF(term, data):
    arrayTF = []
    for i in range(len(term)):
        row = []
        for j in range(len(data)):
            temp = re.findall(term[i],data[j][0])
            row.append(len(temp))
        arrayTF.append(row)
    return arrayTF

In [6]:
print(rowTF(term, getData()))

Processed 914 lines.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
import math

def count(arrayData, term, kelas):
    result = 0
    for i in range(len(arrayData)):
        if arrayData[i][1] == kelas:
            result += len(re.findall(term, arrayData[i][0]))
    return result

def countKelas(arrayData, kelas, arrayTF):
    result = 0
    for i in range(len(arrayData)):
        if arrayData[i][1] == kelas:
            for j in range(len(arrayTF)):
                result += arrayTF[j][i]
    return result

def Pa(arrayProb, arrayTerm, termBaru, indexKelas):
    result = 0
    for i in range(len(arrayTerm)):
        for j in range(len(termBaru)):
            if arrayTerm[i] == termBaru[j]:
#                 print(arrayProb[i][indexKelas])
                result += arrayProb[i][indexKelas]
    return result

def P(arrayProb, arrayTerm, termBaru, indexKelas):
    result = 1
    for i in range(len(termBaru)):
        for j in range(len(arrayTerm)):
            if arrayTerm[j] == termBaru[i]:
#                 print(arrayProb[j][indexKelas])
                result *= arrayProb[j][indexKelas]
#                 print(result)
    return result

def getKelas(arrayData):
    kelas = []
    for i in range(len(arrayData)):
        kelas.append(arrayData[i][1])
    return sorted(list(set(kelas)))
    
def probLikelihood(arrayData, arrayTerm): #probability likelihood P(x|kategori)
    P = []
    TF = rowTF(arrayTerm, arrayData)
    kelas = getKelas(arrayData)
    for i in range(len(arrayTerm)):
        row = []
        for j in range(len(kelas)):
            row.append((count(arrayData, arrayTerm[i], kelas[j])+1)/(countKelas(arrayData, kelas[j], TF)+len(TF)))
        P.append(row)
    return P

def posterior(arrayData, prob, dataBaru, arrayTerm):
    kelas = getKelas(arrayData)
    result = []
    for i in range(len(kelas)):
#         print(Pa(prob, arrayTerm, dataBaru, i))
#         print(P(prob, arrayTerm, dataBaru, i))
        result.append(Pa(prob, arrayTerm, dataBaru, i) * P(prob, arrayTerm, dataBaru, i))
#         print("\n")
    return result

def cekKelas(post, arrayData):
    kelas = getKelas(arrayData)
    max = 0
    index = 0
    for i in range(len(post)):
        if post[i] > max:
            max = post[i]
            index = i
    return kelas[index]

def hitungTingkatAkurasi(uji, postKelas):
    result = 0
    for i in range(len(uji)):
        if uji[i]["label"] == postKelas[i]:
            result += 1
    return result/len(uji) * 100

    

In [8]:
dataTraining = getData()

uji = openFileCsv('data_uji.csv')
dataUji = []
for i in range(len(uji)):
    dataUji.append(ppy.generateTerm(uji[i]["Teks"]))
    
prob = probLikelihood(dataTraining, term)

# for i in range(len(dataUji)):
#     print(dataUji[i])
# print("\n")

kelas = []
for i in range(len(dataUji)):
    post = posterior(dataTraining, prob, dataUji[i], term)
    kelas.append(cekKelas(post, dataTraining))
#     print("data ke %d -> %s" % (i+1, cekKelas(post, dataTraining)))
    
print("\nTinngkatAkurasi = %f%%" %hitungTingkatAkurasi(uji, kelas))


Processed 914 lines.
Processed 228 lines.

TinngkatAkurasi = 87.719298%
